<a href="https://colab.research.google.com/github/MoizAhmed2517/Deep_Learning_Projects/blob/main/Food_Classification_Using_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning - Food Vision App

In [1]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-11-23 18:32:27--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.69.128, 173.194.79.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  40.0MB/s    in 4.8s    

2022-11-23 18:32:32 (33.6 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [2]:
import zipfile

zipref = zipfile.ZipFile("10_food_classes_10_percent.zip")
zipref.extractall()
zipref.close()

In [3]:
import os
for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'")

There are 2 directories and 0 images in '10_food_classes_10_percent'
There are 10 directories and 0 images in '10_food_classes_10_percent/train'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_ric

In [4]:
import numpy as np
import pandas as pd
import pathlib
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Activation, MaxPool2D

In [5]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               class_mode="categorical",
                                               batch_size=BATCH_SIZE)

test_data = train_datagen.flow_from_directory(test_dir,
                                               target_size=IMAGE_SHAPE,
                                               class_mode="categorical",
                                               batch_size=BATCH_SIZE)

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


## Setting our callbacks (things to run whilst our model trains)

Callbacks are extra fucntionality you can add to your models to be performed during or after training. Some of the most popular callbacks are:

* Tracking experiments with the TensorBoard callback
* Model checkpoint with the ModelCheckPoint callback
* Using Earlystopping technique as callback

In [6]:
# Creating a function for settingUp Tensorboard callback

import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-$H%M%S")
  tensorboar_callback = tf.keras.callbacks.TensorBoard(log_dir)
  print(f"Saving Tensorboard log files to:  {log_dir}")
  return tensorboar_callback

In [9]:
# Browsing the tensorflow HUb page and sorting for image classification
# Using following two models in contrast with each other

# 1. ImageNet/Resenet-V2-50/feature vector :https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5
# 2. EfficientNet/B0/feature Vector :https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

In [13]:
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [12]:
import tensorflow_hub as hub

# Creating model builder function

def create_model(model_url, num_classes=10):
  """
  Takes a Tensorfloe HUB URL and creates a Keras Sequential mode with it.

  ArgsL
    model_url (Str): A tensorflow HUB Feature extraction URL
    num_classes (int): NUmber of output neuron in the output layer
      should be equal to number of target classes, default 10

  Returns:
    An uncompile keras sequential model with model_url as feature extractor layer and Dense output layer with num_classes output neuron.
  
  """

  feature_exctractor_layer = hub.KerasLayer(model_url, 
                                            trainable=False,  # Freeze the already learned pattern 
                                            name="feature_extraction_layer",
                                            input_shape=IMAGE_SHAPE+(3,)) 
  
  model = tf.keras.Sequential([
      feature_exctractor_layer,
      Dense(num_classes, activation="softmax", name="output_layer")
  ])

  return model
  


## Creating ResNet TensorFlow HUB Feature Extraction mdoel

In [14]:
resnet_model = create_model(resnet_url,
                            num_classes=train_data.num_classes)

In [17]:
resnet_model.compile(loss="categorical_crossentropy",
                     optimizer=Adam(),
                     metrics=["accuracy"])

In [21]:
resenet_history = resnet_model.fit(train_data,
                 epochs=5,
                 steps_per_epoch=len(train_data),
                 validation_data=test_data,
                 validation_steps=len(test_data),
                 callbacks=[create_tensorboard_callback(dir_name="tensorflow_hub",
                                                        experiment_name="resnet50V2")])

Saving Tensorboard log files to:  tensorflow_hub/resnet50V2/20221123-$H4352
Epoch 1/5
24/24 [==============================] - 18s 749ms/step - loss: 0.2443 - accuracy: 0.9573 - val_loss: 0.6340 - val_accuracy: 0.7900
Epoch 2/5
24/24 [==============================] - 17s 725ms/step - loss: 0.2035 - accuracy: 0.9773 - val_loss: 0.6388 - val_accuracy: 0.7932
Epoch 3/5
24/24 [==============================] - 17s 748ms/step - loss: 0.1738 - accuracy: 0.9827 - val_loss: 0.6232 - val_accuracy: 0.7940
Epoch 4/5
24/24 [==============================] - 17s 716ms/step - loss: 0.1484 - accuracy: 0.9933 - val_loss: 0.6266 - val_accuracy: 0.7948
Epoch 5/5
24/24 [==============================] - 17s 719ms/step - loss: 0.1300 - accuracy: 0.9960 - val_loss: 0.6289 - val_accuracy: 0.7920
